# Item 25: Initialize Parent Classes with super

- The old way to initialize a parent class from a child class is to directly call the parent calss's \_\_init\_\_ mehtod with the child instance.

In [1]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

- This approach works fine for simple hierarchies but breaks down in many cases.
- If your class is affected by multiple inheritance (something to avoid in general), calling the superclasses' \_\_init\_\_ mehotds directly can lead to unpredictable behavior.

- One problem is that the \_\_init\_\_ call order isn't specified across all subclasses. For example, here I define two parent classes that operate on the instance's value field:

In [3]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
    
    
class PlusFive(object):
    def __init__(self):
        self.value += 5

- This class defines its parent classes in one ordering.

In [4]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

- And constructing it produces a result that matches the parent class ordering.

In [5]:
foo = OneWay(5)
print('First ordering is (5 * 2) + 5 =', foo.value)

First ordering is (5 * 2) + 5 = 15


- Here's another class that defines the same parent classes but in a different ordering:

In [12]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

- However, I left the calls to the parent class constructors PlusFive.\_\_init\_\_ and TimesTwo.\_\_init\_\_ in the same order as before, causing this class's behavior not to match the order of the parent classes in its definition.

In [13]:
bar = AnotherWay(5)
print('Second ordring still is', bar.value)

Second ordring still is 15


- Another problem occurs with diamond inheritance. Diamond inheritance happens when a subclass inherits from two separate classes that have the same superclass somewhere in the hierarchy. Diamond inheritance causes the commom superclss's \_\_init\_\_ method to run multiple times, causing unexpected behavior. For example, here I define two child classes that inherit from MyBaseClass.

In [14]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
    
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2    

- Then, I define a child class that inherits from both of these classes, making MyBaseClass the top of the diamond.

In [15]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)
        

foo = ThisWay(5)
print('Should be (5 * 5) + 2 = 27 but is', foo.value)

Should be (5 * 5) + 2 = 27 but is 7


- The output should be 27 because (5 * 5) + 2 = 27. But the call to the second parent class's constructor, PlusTwo.\_\_init\_\_, causes self.value to be reset back to 5 when MyBaseClass.\_\_init\_\_ gets called a second time.

- To solve these problems, Python 2.2 added the super built-in function and defined the method resolution order (MRO). The MRO standardizes which superclasses are initialized before orthers.

- Here, I create a diamond-shaped class hierarchy again, but this time I use super (in the Python 2 style) to initialize the parent class.

In [16]:
# Python 2
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        

class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2

- Now the top part of the diamond, MyBaseCalss.\_\_init\_\_, is only run a single time. The other parent classes are run in the order specified in the class statement.

In [17]:
# Python 2
class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)

In [18]:
foo = GoodWay(5)
print('Should be 5 * (5 + 2) = 35 and is', foo.value)

Should be 5 * (5 + 2) = 35 and is 35


- This order may seem backwards at first. Shouldn't TimesFiveCorrect.\_\_init\_\_ have unr first? Shouldn't the result be (5 * 5) + 2 = 27? The answer is no. This ordering matches what the MRO defines for this class. The MRO ordering is available on a class method called mro.

In [20]:
from pprint import pprint
pprint(GoodWay.mro())

[<class '__main__.GoodWay'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


- When I call GoodWay(5), it in turn calls TimesFiveCorrect.\_\_init\_\_, which calls PlusTwoCorrect.\_\_init\_\_, which calls MyBaseClass.\_\_init\_\_. Once this reaches the top of the diamond, then all of the initialization methods actually do their work in the opposite order from how their \_\_init\_\_ functions were called.

- MyBaseClass.\_\_init\_\_ assigns the value to 5. PlusTwoCorrect.\_\_init\_\_ adds 2 to make value equal 7. TimesFiveCorrect.\_\_init\_\_ multiplies it by 5 to make value equal 35.

- The super built-in function works well, but it still has two noticeable problems in Python 2:

- Its syntax is a nit verbose. You ahve to specify the class you're in, the self object, the method name(usually \_\_init\_\_), and all the arguments. This construction can be confusing to new Python programmers.
- You have to specify the current class by name in the call to super. If you ever change the class's name-a very common activity when improving a class hierarchy - you also need to update every call to super.

- Thankfully, Python 3 fixes these issues by making calls to super with no arguments equivalent to calling super with \_\_class\_\_ and self specified. In Python 3, you should always use super because it's clear, concise, and always does the right thing.

In [21]:
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)
        
    
class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        
assert Explicit(10).value == Implicit(10).value

- This works because Python 3 lets you reliably reference the current class in methods using the \_\_class\_\_ variablew. This doesn't work in Python 2 because \_\_class\_\_ isn't defined. You may guess that you could use self.\_\_class\_\_ as an argument to super, but this breaks because of the way super is implemented in Python 2.

## Things to Remember

- Python's standard method resolution order (MRO) solves the problems of superclass initialization order and diamond inheritance.
- Always use the super built-in function to initialize parent classes.

# Item 26: Use Multiple Inheritance Only for Mix-in Utility Classes

- Python is an object-oriented language with built-in facilities for making multiple inheritance tractable. However, it's better to avoid multiple inheritance altogether.

- If you find yourself desiring the convenience and encapsulation that comes with multiple inheritance, consider wrinting a *mix-in* instead. A mix-in is a small class that only define their own instance attributes nor require their \_\_init\_\_ constructor to be called.

- Writing mix-ins is easy because Python makes it trivial to inspect the current state of any object regardless of its type. Dynamic inspection lets you write generic functionality a single time, in a mix-in, that can be applied to many other classes. Mix-ins can be composed and layered to minimize repetitive code and maximize reuse.

- For example, say you want the ability to convert a Python object from its in-memory representation to a dictionary so you can use it with all of your classes?

- Here, I define an example mix-in that accomplishes this with a new public method that's added to any class that inherits from it:

In [26]:
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

- The implementation details are straightforward and rely on dynamic attribute access using hasattr, dynamic type inspection with isinstance, and accessing the instance dictionary \_\_dict\_\_.

- Here, I define an example class that uses the mix-in to make a dictionary representation of a binary tree:

In [27]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

- Translating a large number of related Python objects into a dictionary becomes easy.

In [28]:
tree = BinaryTree(10,
                  left=BinaryTree(7, right=BinaryTree(9)),
                  right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


- The best part about mix-ins is that you can make their generic functionality pluggable so behaviors can be overridden when required. For example, here I define a subclass of BinaryTree that holds a reference to tis parent. This circular reference would cause the default implementation of ToDictMixin.to\_dict to loop forever. 

In [39]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None,
                right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
        
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and 
               key == 'parent'):
            return value.value 
        else:
            return super()._traverse(key, value)

- The solution is to override the ToDictMixin.\_traversee method in thhe BinaryTreeWithParent class to only process values that matter, preventing cycles encountered by the mix-in. Here, I override the \_traverse method to not traverse the parent and just dinsert its numerical value:

- Calling BinaryTreeWithParent.to\_dict will work without issue because the circular referencing properties aren't followed.

In [40]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None, 'parent': 7}, 'parent': 10}, 'right': None, 'parent': None}


- By defining BinaryTreeWithParent.\_traverse, I've also enabled any class that has an attribute of tpye BinaryTreeWithParent to automatically work with ToDictMixin.

In [41]:
class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent
        
my_tree = NamedSubTree('foobar', root.left.right)
print(my_tree.to_dict())

{'name': 'foobar', 'tree_with_parent': {'value': 9, 'left': None, 'right': None, 'parent': 7}}


- Mix- ins can also be composed together. For example, say you want a mix-in that provides generic JSON serialization for any class. You can do this by assuming that a class provides a to\_dict method( which may or may not be provided by the ToDictMixin class),

In [42]:
class JsonMixin(object):
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

- Note how the JsonMixin class defines both instance methods and class methods. Mix-ins let you add either kind of behavior. In this example, the only requirements of the JsonMixin are that the class has a to\_dict method and its \_\_init\_\_ method takes keyword arguments.

- This mix-in makes it simple to create hierarchies of utility classes that can be serialized to and from JSON with little boilerplate. For example, here I have a hierarchy of data classes representing part of a datacenter topology:

In [43]:
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [
            Machine(**kwargs) for kwargs in machines]
    
class Switch(ToDictMixin, JsonMixin):
    #...
    
    
class Machine(ToDictMixin, JsonMixin):
    # ....

IndentationError: expected an indented block (<ipython-input-43-bab5f43f7f3a>, line 11)

- Serializing these claases to and from JSON is simple. Here, I verify that the data is able to be sent round-trip through serializing and deserializing:

In [44]:
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 8, "ram": 32e9, "disk": 5e12},        
    ]
}"""

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()
assert json.loads(serialized) == json.loads(roundtrip)

- When you use mix-ins like this, it's also fine if the class already inherits from JsonMixin higher up in the object hierarchy. The resulting class will behave the same way.

## Things to Remember

- Avoid using multiple inheritance if mix-in classes can achieve the same outcome.
- Use pluggable behaviors at the instance level to provide per-class customization when mix-in classes may require it.
- Compose mix-ins to create complex functionality from simple behaviors